In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions  import from_unixtime
from pyspark.sql.functions  import to_date
from pyspark.sql import Row
from pyspark.sql.functions import to_json, struct
from pyspark.sql import functions as F
import random
import time

In [0]:
# Config details for Azure SQL DB for VehicleInformation and LocationInformation tables
sqldbusername = dbutils.secrets.get(scope="KeyVaultScope",key="VehicleInformationDBUserId")
sqldbpwd=dbutils.secrets.get(scope="KeyVaultScope",key="VehicleInformationDBPwd")

jdbcHostname = "vehicleinformatiosrvr.database.windows.net"
jdbcDatabase = "VehicleInformationDB"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, sqldbusername, sqldbpwd)
connectionProperties = {
  "user" : sqldbusername,
  "password" : sqldbpwd,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
# Reading dbo.VehicleInfo master table from Azure SQL DB
vehicleInfo = "(select VehicleId,Make,Model,Category,ModelYear from dbo.VehicleInformation) vehilce"
df_vehicleInfo = spark.read.jdbc(url=jdbcUrl, table=vehicleInfo, properties=connectionProperties)
display(df_vehicleInfo)

VehicleId,Make,Model,Category,ModelYear
04ac43cf-ed3c-4fb7-a15e-2dabd3c8342e,Mitsubishi,3000GT,Hatchback,1999
115e6d02-4a43-4131-b6af-9d2a8d642073,BMW,M Series,Convertible,2013
2832b3ec-222c-4049-9af5-45a8d66d6b58,MAZDA,323,Hatchback,1992
288e3ee8-ea29-489b-9c3d-07e7c6d6ee99,Toyota,4Runner,SUV,2013
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,BMW,1 Series,"Coupe, Convertible",2013
5cb4ae8e-19de-40e1-96b5-874ffc43210b,Saab,3-Sep,"Hatchback, Convertible",1999
90ccea91-416e-453f-b582-e6f04f02ee40,Toyota,2500 Crew Cab,Pickup,2021
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,Chevrolet,2500 Extended Cab,Pickup,1992
b51480fc-af07-491d-b914-fec46d7d7d47,Chevrolet,300 CE,Coupe,1992
c79935cc-0b88-44ae-9765-a68d3c0fd37d,Chrysler,300,Sedan,2017


In [0]:
df_vehicleInfo.createOrReplaceTempView("vw_VehicleMaster")

In [0]:
# Reading dbo.LocationInfo master table from Azure SQL DB and creating a view

locationInfo = "(select Borough,Location,Latitude,Longitude from dbo.LocationInfo) vehilce"
df_locationInfo = spark.read.jdbc(url=jdbcUrl, table=locationInfo, properties=connectionProperties)

df_locationInfo.createOrReplaceTempView("vw_LocationMaster")
display(df_locationInfo)

Borough,Location,Latitude,Longitude
MANHATTAN,339 East 12th Street,40.73061,-73.984016
BROOKLYN,185 Erasmus Street,40.650002,-73.949997
STATEN ISLAND,630 Richmond Hill Road,40.579021,-74.151535
BRONX,1475 Thieriot Avenue,40.837048,-73.865433
QUEENS,"Thrilla, New York",40.742054,-73.769417


In [0]:
storageAccount="cookbookstoragegen2"
mountpoint = "/mnt/SensorData"
storageEndPoint ="abfss://sensordata@{}.dfs.core.windows.net/".format(storageAccount)
print ('Mount Point ='+mountpoint)

#ClientId, TenantId and Secret is for the Application(ADLSGen2App) was have created as part of this recipe
clientID ="17fbb29f-fff9-4841-99e5-827aea6c0161"
tenantID ="72f988bf-86f1-41af-91ab-2d7cd011db47"
clientSecret ="E-fE3UEPAeH.VIBT8AKm6c2WX9QF4-a._1"
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)


configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientID,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}

try:
  dbutils.fs.mount(
  source = storageEndPoint,
  mount_point = mountpoint,
  extra_configs = configs)
except:
    print("Already mounted...."+mountpoint)


Mount Point =/mnt/SensorData
Already mounted..../mnt/SensorData

In [0]:
# dbutils.fs.unmount("/mnt/SensorData")

In [0]:
display(dbutils.fs.ls("dbfs:/mnt/SensorData//"))

path,name,size
dbfs:/mnt/SensorData/historicalvehicledata/,historicalvehicledata/,0
dbfs:/mnt/SensorData/vehiclestreamingdata/,vehiclestreamingdata/,0


In [0]:
#Creating the schema for the vehicle data json structure
jsonschema = StructType() \
.add("id", StringType()) \
.add("eventtime", TimestampType()) \
.add("rpm", IntegerType()) \
.add("speed", IntegerType()) \
.add("kms", IntegerType()) \
.add("lfi", IntegerType())  \
.add("lat", DoubleType()) \
.add("long", DoubleType())

In [0]:
#Function to create required folders in mount point
def checkpoint_dir(type="Bronze"): 
  val = f"/mnt/SensorData/vehiclestreamingdata/{type}/chkpnt/" 
  return val

def delta_dir(type="Bronze"): 
  val = f"/mnt/SensorData/vehiclestreamingdata/{type}/delta/" 
  return val

def hist_chkpt_dir(type="Hist"): 
  val = f"/mnt/SensorData/historicalvehicledata/{type}/chkpnt" 
  return val
 
def hist_dir(type="Hist"): 
  val = f"/mnt/SensorData/historicalvehicledata/{type}/data" 
  return val
 

In [0]:
#Event Hubs for Kafak configuration details
BOOTSTRAP_SERVERS = "kafkaenabledeventhubns.servicebus.windows.net:9093"
EH_SASL = "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"Endpoint=sb://kafkaenabledeventhubns.servicebus.windows.net/;SharedAccessKeyName=sendreceivekafka;SharedAccessKey=4vxbVwOGJD7b4aVcUWBvYp440+SFHpRyQVIpMeXvoVE=\";"
GROUP_ID = "$Default"


In [0]:
# Function to read data from EventHub and writing as delta format
def append_kafkadata_stream(topic="vehiclesensoreventhub"):
  kafkaDF = (spark.readStream \
    .format("kafka") \
    .option("subscribe", topic) \
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.jaas.config", EH_SASL) \
    .option("kafka.request.timeout.ms", "60000") \
    .option("kafka.session.timeout.ms", "60000") \
    .option("kafka.group.id", GROUP_ID) \
    .option("failOnDataLoss", "false") \
    .option("startingOffsets", "latest") \
    .load().withColumn("source", lit(topic)))
  
  newkafkaDF=kafkaDF.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)","source").withColumn('vehiclejson', from_json(col('value'),schema=jsonschema))
  kafkajsonDF=newkafkaDF.select("key","value","source", "vehiclejson.*")
  
  query=kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"eventtime"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" \
                  ,"lfi" \
                  ,"lat" \
                  ,"long" \
                  ,"source") \
            .writeStream.format("delta") \
            .outputMode("append") \
            .option("checkpointLocation",checkpoint_dir("Bronze")) \
            .start(delta_dir("Bronze")) 
 
  return query

In [0]:
# Function to read data from ADLS gen-2 using readStream API and writing as delta format
def append_batch_source():
  
  topic ="historical"

  kafkaDF = (spark.readStream \
     .schema(jsonschema)
    .format("parquet") \
    .load(hist_dir("Hist")).withColumn("source", lit(topic)))
  
  query=kafkaDF.selectExpr(
                  "id"	  \
                  ,"eventtime"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms" \
                  ,"lfi" \
                  ,"lat" \
                  ,"long" \
                  ,"source"
                 ) \
            .writeStream.format("delta") \
            .option("checkpointLocation",checkpoint_dir("Hist")) \
            .outputMode("append") \
            .start(delta_dir("Hist")) 
 
  return query

In [0]:
# Reading data from EventHubs for Kafka
query_source1 = append_kafkadata_stream(topic='vehiclesensoreventhub')


In [0]:
# Reading data from Historical location ( in this example its from ADLS Gen-2 having historical data for Vehicle Sensor.)
# There may be cases where historical data can be added to this location from any other source where the schema is same for all the files. IN such scenarios using readStream API on Gen-2 location will keep polling for new data and when available it will be ingested
query_source2 = append_batch_source()

In [0]:
# Dropping all Delta tables if required
def DropDeltaTables(confirm=1):
  
  if(confirm ==1):
    spark.sql("DROP TABLE IF EXISTS VehicleSensor.VehicleDelta_Bronze")
    spark.sql("DROP TABLE IF EXISTS VehicleSensor.VehicleDelta_Silver")
    spark.sql("DROP TABLE IF EXISTS VehicleSensor.VehicleDeltaAggregated")
    spark.sql("DROP TABLE IF EXISTS VehicleSensor.VehicleDelta_Historical")
    

In [0]:
#Function which drops all delta tables. TO avoid droping tables call the function with confirm=0
DropDeltaTables(confirm=0)

In [0]:
%sql
-- Creating the delta table on delta location for Bronze data
CREATE DATABASE IF NOT EXISTS VehicleSensor;

In [0]:
#Create historical delta table
spark.sql("CREATE TABLE IF NOT EXISTS VehicleSensor.VehicleDelta_Historical USING DELTA LOCATION '{}'".format(delta_dir("Hist")))

Out[55]: DataFrame[]

In [0]:
# Wait for 5 seconds before we create the delta tables else it might error out that delta location is not created
time.sleep(5) 

###  Creating Bronze Delta Table

In [0]:
%sql
-- Creating the delta table on delta location for Bronze data
CREATE DATABASE IF NOT EXISTS VehicleSensor;
CREATE TABLE IF NOT EXISTS VehicleSensor.VehicleDelta_Bronze
USING DELTA
LOCATION "dbfs:/mnt/SensorData/vehiclestreamingdata/Bronze/delta"

In [0]:
%sql
describe formatted VehicleSensor.VehicleDelta_Bronze

col_name,data_type,comment
id,string,
eventtime,timestamp,
rpm,int,
speed,int,
kms,int,
lfi,int,
lat,double,
long,double,
source,string,
,,


In [0]:
#Streaming Data from Bronze Delta Table. This will help in only extracting new data coming from Event Hubs to be loaded into Silver Delta tables.
df_bronze=spark.readStream.format("delta").option("latestFirst", "true").table("VehicleSensor.VehicleDelta_Bronze")

In [0]:
#Creating Temp View on Bronze DF
df_bronze.createOrReplaceTempView("vw_TempBronze")

In [0]:
%sql
select count(*) from vw_TempBronze

count(1)
722


In [0]:
%sql
-- select count(*),hour(eventtime) as hour, day(eventtime) as day from vw_TempSilver group by hour(eventtime),day(eventtime)
select *, Year(eventtime) as Year, month(eventtime) as Month,day(eventtime) as Day, hour(eventtime) as Hour from vw_TempBronze limit 10

id,eventtime,rpm,speed,kms,lfi,lat,long,source,Year,Month,Day,Hour
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T13:43:39.210+0000,11,94,4765,0,40.650002,-73.949997,vehiclesensoreventhub,2021,7,18,13
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T14:43:39.573+0000,17,105,8985,0,40.837048,-73.865433,vehiclesensoreventhub,2021,7,18,14
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T15:43:39.631+0000,2,75,5555,1,40.579021,-74.151535,vehiclesensoreventhub,2021,7,18,15
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T16:43:39.694+0000,4,116,4008,0,40.579021,-74.151535,vehiclesensoreventhub,2021,7,18,16
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T17:43:39.756+0000,92,80,9309,1,40.650002,-73.949997,vehiclesensoreventhub,2021,7,18,17
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T18:43:39.838+0000,2,102,4289,0,40.650002,-73.949997,vehiclesensoreventhub,2021,7,18,18
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T19:43:39.898+0000,99,76,1187,0,40.837048,-73.865433,vehiclesensoreventhub,2021,7,18,19
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T20:43:39.943+0000,49,86,5688,0,40.837048,-73.865433,vehiclesensoreventhub,2021,7,18,20
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T21:43:39.997+0000,74,111,834,1,40.742054,-73.769417,vehiclesensoreventhub,2021,7,18,21
4b1e829f-5f32-4ae9-b415-9e7f0eb15401,2021-07-18T22:43:40.058+0000,92,118,4481,0,40.579021,-74.151535,vehiclesensoreventhub,2021,7,18,22


In [0]:
#Streaming Data from History Delta Table
df_historical=spark.readStream.format("delta").option("latestFirst", "true").table("VehicleSensor.VehicleDelta_Historical")

In [0]:
#Joining both historical and Bronze Streaming Data
df_bronze_hist = df_bronze.union(df_historical)

In [0]:
df_bronze_hist.createOrReplaceTempView("vw_TempBronzeHistorical")

In [0]:
%sql
select count(*) from vw_TempBronzeHistorical

count(1)
1144


###  Creating Silver Delta Table

In [0]:
# We are always combining data from streaming and batch data by using Structured streaming

df_silver= spark.sql("select s.*,m.Make,m.Model,m.Category, Year(eventtime) as Year, month(eventtime) as Month,day(eventtime) as Day, \
                     hour(eventtime) as Hour,l.Borough,l.Location  \
                     from vw_TempBronzeHistorical s \
                     left join vw_VehicleMaster m on s.id = m.VehicleId \
                     left join vw_LocationMaster l on s.lat = l.Latitude and s.long = l.Longitude") \
            .writeStream.format("delta").option("MergeSchem","True") \
            .outputMode("append") \
            .option("checkpointLocation",checkpoint_dir("Silver"))  \
            .start(delta_dir("Silver"))

In [0]:
# Wait for 3 seconds before we create the delta tables else it might error out that delta location is not created
time.sleep(5) 

In [0]:
%sql
-- drop  TABLE IF  EXISTS VehicleSensor.VehicleDelta_Silver;
CREATE TABLE IF NOT EXISTS VehicleSensor.VehicleDelta_Silver
USING DELTA
LOCATION "dbfs:/mnt/SensorData/vehiclestreamingdata/Silver/delta/"

In [0]:
%sql
select count(*) from VehicleSensor.VehicleDelta_Silver

count(1)
1144


In [0]:
%sql
describe formatted VehicleSensor.VehicleDelta_Silver

col_name,data_type,comment
id,string,
eventtime,timestamp,
rpm,int,
speed,int,
kms,int,
lfi,int,
lat,double,
long,double,
source,string,
Make,string,


###  Creating Gold Delta Table

In [0]:

df_gold=(spark.readStream.format("delta").option("latestFirst", "true").table("VehicleSensor.VehicleDelta_Silver") \
                                 .groupBy(window('eventtime',"1 hour"),"Make","Borough","Location","Month","Day","Hour").count()) \
                                 .writeStream.format("delta") \
                                              .outputMode("complete") \
                                              .option("checkpointLocation",checkpoint_dir("Gold"))  \
                                              .start(delta_dir("Gold"))


In [0]:
time.sleep(10) 

In [0]:
#Create VehicleDeltaAggregated delta table

spark.sql("CREATE TABLE IF NOT EXISTS VehicleSensor.VehicleDeltaAggregated USING DELTA LOCATION '{}'".format(delta_dir("Gold")))

Out[66]: DataFrame[]

In [0]:
%sql
-- select max(eventtime) from VehicleSensor.VehicleDelta_Silver where eventtime between '2021-05-06T16:00:46.905+0000' and '2021-05-06T17:00:00.905+0000'--id='aa7e09d0-92cb-4cc0-99e2-04602ab0f85a'  order by Eventtime desc limit 10;

-- select * from  VehicleSensor.VehicleDelta_Bronze  where id='aa7e09d0-92cb-4cc0-99e2-04602ab0f85a' order by Eventtime desc limit 10;

select * from  VehicleSensor.VehicleDelta_Silver  where id='aa7e09d0-92cb-4cc0-99e2-04602ab0f85a' order by Eventtime desc limit 10;


id,eventtime,rpm,speed,kms,lfi,lat,long,source,Make,Model,Category,Year,Month,Day,Hour,Borough,Location
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-19T02:55:58.137+0000,63,98,8213,0,40.742054,-73.769417,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,19,2,QUEENS,"Thrilla, New York"
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-19T02:43:43.586+0000,1,97,1428,0,40.742054,-73.769417,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,19,2,QUEENS,"Thrilla, New York"
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-19T01:55:58.055+0000,33,97,8616,1,40.742054,-73.769417,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,19,1,QUEENS,"Thrilla, New York"
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-19T01:43:43.538+0000,64,98,8042,0,40.837048,-73.865433,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,19,1,BRONX,1475 Thieriot Avenue
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-19T00:55:58.011+0000,28,112,9919,0,40.650002,-73.949997,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,19,0,BROOKLYN,185 Erasmus Street
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-19T00:43:43.494+0000,11,74,115,0,40.579021,-74.151535,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,19,0,STATEN ISLAND,630 Richmond Hill Road
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-18T23:55:57.971+0000,61,102,1956,0,40.837048,-73.865433,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,18,23,BRONX,1475 Thieriot Avenue
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-18T23:43:43.448+0000,54,79,2356,1,40.837048,-73.865433,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,18,23,BRONX,1475 Thieriot Avenue
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-18T22:55:57.901+0000,98,102,4476,0,40.837048,-73.865433,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,18,22,BRONX,1475 Thieriot Avenue
aa7e09d0-92cb-4cc0-99e2-04602ab0f85a,2021-07-18T22:43:43.400+0000,99,117,5401,0,40.73061,-73.984016,vehiclesensoreventhub,Chevrolet,2500 Extended Cab,Pickup,2021,7,18,22,MANHATTAN,339 East 12th Street


In [0]:
df_silver_agg=(spark.readStream.format("delta").table("VehicleSensor.VehicleDelta_Silver"))
df_silver_agg.createOrReplaceTempView("vw_AggDetails")

In [0]:
%sql
-- Viwing data from the Gold Delta Tables
select * from VehicleSensor.VehicleDeltaAggregated
ORDER BY Month DESC, Day Desc,count desc  limit 10

window,Make,Borough,Location,Month,Day,Hour,count
"List(2020-11-06T03:00:00.000+0000, 2020-11-06T04:00:00.000+0000)",Saab,null,null,11,6,3,1
"List(2020-11-05T03:00:00.000+0000, 2020-11-05T04:00:00.000+0000)",Saab,null,null,11,5,3,2
"List(2020-11-05T03:00:00.000+0000, 2020-11-05T04:00:00.000+0000)",Chrysler,null,null,11,5,3,1
"List(2020-11-04T03:00:00.000+0000, 2020-11-04T04:00:00.000+0000)",Saab,null,null,11,4,3,2
"List(2020-11-04T03:00:00.000+0000, 2020-11-04T04:00:00.000+0000)",Chrysler,null,null,11,4,3,1
"List(2020-11-04T03:00:00.000+0000, 2020-11-04T04:00:00.000+0000)",BMW,null,null,11,4,3,1
"List(2020-11-03T03:00:00.000+0000, 2020-11-03T04:00:00.000+0000)",BMW,null,null,11,3,3,2
"List(2020-11-03T03:00:00.000+0000, 2020-11-03T04:00:00.000+0000)",Saab,null,null,11,3,3,2
"List(2020-11-03T03:00:00.000+0000, 2020-11-03T04:00:00.000+0000)",Chrysler,null,null,11,3,3,1
"List(2020-11-02T03:00:00.000+0000, 2020-11-02T04:00:00.000+0000)",BMW,null,null,11,2,3,2


In [0]:
# server_name = "vehicleinformatiosrvr.database.windows.net"
# database_name = "VehicleInformationDB"
# url = server_name + ";" + "databaseName=" + database_name + ";"

# table_name = "VehicleInformation"
# username = sqldbusernamedd
# password = sqldbpwd
# jdbcDF = spark.read \
#         .format("com.microsoft.sqlserver.jdbc.spark") \
#         .option("url", url) \
#         .option("dbtable", table_name) \
#         .option("user", username) \
#         .option("password", password).load()

In [0]:
# jdbcDF.load()

In [0]:
writeConfig = {
    "Endpoint": "https://vehicleinformationdb.documents.azure.com:443/",
    "Masterkey": "woZwUr9FXx50eIZBDyzSSMOyAuuSLy5NJXbFdx1oEYyr7gFjoemtk51aoNN50DJFGz0rVXv5fJRnAB0jkmybnA==",
    "Database": "vehicle",
    "Collection": "vehicleinformation",
    "Upsert": "true",
    "WritingBatchSize": "500"
   }